In [53]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
from tensorflow.python.framework import ops
import tflearn
import random

In [54]:
!pip install tflearn

In [55]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [56]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [57]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [58]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [59]:
words=[]
documents=[]
classes=[]
ignore_words=['?']

stemmer = LancasterStemmer()

# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
     # tokenize each word in the sentence
         w=nltk.word_tokenize(pattern)
     # add to our words list
         words.extend(w)
    # add to documents in our corpus
         documents.append((w, intent['tag']))
     # add to our classes list
         if intent['tag'] not in classes:
             classes.append(intent['tag'])
        

# stem and lower each word and remove duplicates
words=[stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))

# remove duplicates
classes=sorted(list(set(classes)))

In [60]:
print (len(documents), "documents",documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)


25 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Namaste'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['TELL', 'ME', 'ABOUT', 'TAKING', 'SOIL', 'SAMPLE'], 'Soil Testing'), (['How', 'to', 'take', 'soil', 'sample', '?'], 'Soil Testing'), (['What', 'is', 'the', 'technique', 'used', 'in', 'taking', 'soil', 'sample'], 'Soil Testing'), (['TELL', 'ME', 'SOMETHING', 'ABOUT', 'NUTRIENT', 'MANAGEMENT', '?'], 'Nutrient Management'), (['What', 'are', 'some', 'ways', 'to', 'improve', 'nutrients', 'in', 'soil'], 'Nutrient Management'), (['How', 'to', 'manage', 'nutrients'], 'Nutrient Management'), (['TELL', 'ME', 'HOW', 'TO', 'CONTROL', 'INSECT', 'IN', 'WHEAT'], 'Plant Protection'), (['What', 'are', 'some', 'ways', 'to', 'co

In [61]:
# create our training data
training=[]
output=[]

# create an empty array for our output
output_empty=[0]*len(classes)

# training set, bag of words for each sentence
for doc in documents:
    bag=[]
     # list of tokenized words for the pattern
    pattern_words=doc[0]
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
     # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
     # output is a '0' for each tag and '1' for current tag
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
# shuffle our features and turn into np.array.
random.shuffle(training)
training=np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


/var/folders/f8/pr3njbwj3vl3_thlf_2n0njr0000gp/T/ipykernel_51117/45723145.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [62]:
# reset underlying graph data
ops.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00869 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.00869 - acc: 1.0000 -- iter: 24/25
Training Step: 4000  | total loss: 0.00839 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.00839 - acc: 1.0000 -- iter: 25/25
--
INFO:tensorflow:/Users/aditya/Downloads/Farmer-friendly-contextual-chatbot-master/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [63]:
# save all of our data structures
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open('training_data','wb'))